In [1]:
FileLabelName = '../../../Data/Dog/labels.csv'
FileImageName = '../../../Data/Dog/train/'
import zipfile

# z = zipfile.ZipFile(FileLabelName, 'r') # 这里的第二个参数用r表示是读取zip文件，w是创建一个zip文件

# for f in z.namelist():
#     bytes = z.read(f)

In [2]:
DEBUG = False
Size = 224

In [3]:
import pandas as pd
import os
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy.misc import imresize
from sklearn.utils import shuffle

import sys
sys.path.append('../CIFAR-10/')
import Net_Train

In [4]:
Label = pd.read_csv(FileLabelName)
#Label = shuffle(Label)
print(len(Label))
Label = Label.sample(frac=1).reset_index(drop = True)
#print(Label['id'][17], Label['breed'][17])

10000


In [5]:
i = 0
Image_Data = []
for root, dirs, files in os.walk(FileImageName):
    # print(root)
    for file in FileImageName+Label['id']:
        #print(file)
        img = mpimg.imread(file + '.jpg')
        #print(img.shape)
        img = imresize(img, (Size,Size))
        img = img.transpose((2,1,0))
        #print(img.shape)
        img = img.flatten()
#         print(img.shape)
#         img = img.reshape((Size, Size, 3))
#         plt.imshow(img)
#         plt.title(Label['breed'][i])
#         plt.show()
        Image_Data.append(img)
        i += 1
        if DEBUG:
            if i>17:
                break
#print(Image_Data)
Image_Data = np.array(Image_Data)
print(Image_Data.shape)

/home/ubuntu/Yongchao/yongchao/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if __name__ == '__main__':
/home/ubuntu/Yongchao/yongchao/lib/python3.6/site-packages/scipy/misc/pilutil.py:554: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  im = toimage(arr, mode=mode)
/home/ubuntu/Yongchao/yongchao/lib/python3.6/site-packages/scipy/misc/pilutil.py:380: DeprecationWarning: `bytescale` is deprecated!
`bytescale` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
  bytedata = bytescale(data, high=high, low=low, cmin=cmin, cmax=cmax)
/home/ubuntu/Yongchao/yongchao/lib/python3.6/site-packages/scipy/misc/pilutil.py:565: DeprecationWarning: `fromimage` is deprecated!
`fromimage` is deprecated in SciPy 1

(10000, 150528)


In [6]:
ImageLabel = np.array(Label['breed']) #.reshape(-1, 1)
#print(ImageLabel)
encoder = preprocessing.LabelEncoder()
ImageLabel = encoder.fit_transform(ImageLabel)
ImageLabel = ImageLabel.reshape((1,-1))
Label_inverse = np.array(list(encoder.classes_))
Classes = len(Label_inverse)
print(Classes)
print(ImageLabel.shape)
Data = np.concatenate((ImageLabel.T, Image_Data), axis=1)
print(Data.shape)

120
(1, 10000)
(10000, 150529)


In [7]:
Train_data, Test_data = Net_Train.loadTrainTestData(Data, 200, Train_ratio=0.9, origin_size=224)#, resize=224)

9000 1000


In [ ]:
print(Train_data.shape, Test_data.shape)

In [ ]:
from mxnet.gluon.model_zoo import vision
from mxnet import gluon
from mxnet import init
import mxnet
AlexNet = vision.resnet18_v2(pretrained=True)
print(AlexNet)
# AlexNet.initialize(init=init.Xavier(), ctx = mxnet.gpu())
Fine_Tuning = vision.resnet18_v2(classes = 120)
Fine_Tuning.features = AlexNet.features
Fine_Tuning.classifier.initialize(init=init.Xavier())
Fine_Tuning.collect_params().reset_ctx(mxnet.gpu())

loss = gluon.loss.SoftmaxCrossEntropyLoss()
Trainer = gluon.Trainer(Fine_Tuning.collect_params(), 'sgd', {'learning_rate': 1})

In [ ]:
Net_Train.train(Train_data, Test_data, Fine_Tuning, 5, loss, Trainer, 224, ctx=mxnet.gpu(), Test_Flag=True, lr = 1)

## 错误消息导致kernel暂停
1. GPU上数据太多，导致GPU内存爆满，直接退出

In [ ]:
#Net = "Fine_tuning_resnet18.params"
#Fine_Tuning.collect_params().save(Net)

In [ ]:

#print(Fine_Tuning.keys())

In [1]:
from mxnet import nd
import mxnet
Train_data = nd.ones(shape=(10000,10000), ctx= mxnet.gpu())
print(Train_data)


[[ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]
 ..., 
 [ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]]
<NDArray 10000x10000 @gpu(0)>


In [3]:
Train_data.as_in_context(mxnet.cpu())
#del Train_data
#Train_data = Train_data.asnumpy()


[[ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]
 ..., 
 [ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]]
<NDArray 10000x10000 @cpu(0)>

In [ ]:
print(Train_data)